In [1]:
import sys
sys.path.append('./lib')

from pathlib import Path

import numpy as np
import pandas as pd

from model import PDFmodel, generate_mse_loss

In [2]:
seed = 1231
np.random.seed(seed)

In [3]:
SAVE_DIR = Path('./test_results/model')
SAVE_DIR.mkdir(parents=True, exist_ok=True)

In [4]:
# Collect Tommaso's data
fk_grid = np.load('Tommaso/fk_grid.npy')
data = np.load('Tommaso/data.npy')
FK = np.load('Tommaso/FK.npy')
f_bcdms = np.load('Tommaso/f_bcdms.npy')
Cy = np.load('Tommaso/Cy.npy')
#noise = np.load('Tommaso/L1_noise_BCDMS.npy')

In [5]:
# Prepare index for covariance matrix
arrays = [
  ['T3' for _ in range(Cy.shape[0])],
  ['BCDMS' for _ in range(Cy.shape[0])],
  ]
multi_index = pd.MultiIndex.from_arrays(arrays, names=('group', 'dataset'))
Cinv = pd.DataFrame(np.linalg.inv(Cy), index=multi_index, columns=multi_index)
y = FK @ f_bcdms # L1 data

In [6]:
# ========== Model ==========
mse_loss = generate_mse_loss(Cinv)
pdf = PDFmodel(
                dense_layer='Dense',
                input=fk_grid,
                outputs=1,
                architecture=[28, 25],
                activations=['tanh', 'tanh'],
                kernel_initializer='GlorotNormal',
                user_ki_args=None,
                seed=seed,)
pdf.model.summary()

central_data_dict = {'BCDMS': y}
fk_dict = {'BCDMS': FK}

predictions_0 = pdf.predict()

# ========== Training ==========
pdf.train_network_gd(data=central_data_dict,
                      FK_dict=fk_dict,
                      loss_func=mse_loss,
                      learning_rate=1e-3,
                      tol=0.0,
                      logging=True,
                      callback=True,
                      max_epochs=10,
                      log_fr=5,
                      savedir=SAVE_DIR,
                      optimizer='Adam')

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ deep_layer_0 (Dense)            │ (None, 50, 28)         │            56 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ deep_layer_1 (Dense)            │ (None, 50, 25)         │           725 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 50, 1)          │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 807 (3.15 KB)

 Trainable params: 807 (3.15 KB)

 Non-trainable params: 0 (0.00 B)

Maximum number of iterations reached.


In [7]:
# Compute predictions after training
predictions = pdf.predict().numpy().reshape(-1)

In [8]:
model_end = PDFmodel.load_model(config_path=str(SAVE_DIR / 'config.json'),
                                         weights_path=str(SAVE_DIR / 'epoch_11.weights.h5'))
model_init = PDFmodel.load_model(config_path=str(SAVE_DIR / 'config.json'),
                                 weights_path=str(SAVE_DIR / 'epoch_0.weights.h5'))

In [9]:
# Check predictions are the same at initialization
predictions_loaded_init = model_init(fk_grid).numpy().reshape(-1)
assert(np.allclose(predictions_loaded_init, predictions_0.numpy().reshape(-1)))

In [10]:
# Check predictions are the same at the end of training
predictions_loaded_end = model_end(fk_grid).numpy().reshape(-1)
assert(np.allclose(predictions_loaded_end, predictions.reshape(-1)))

In [11]:
# Check weights are the same
weights = pdf.model.get_weights()
weights_loaded = model_end.model.get_weights()

assert(len(weights) == len(weights_loaded))

for i in range(len(weights)):
    assert(np.allclose(weights[i], weights_loaded[i]))

In [16]:
# Compute the NTK
ntk = pdf.compute_ntk()
ntk_loaded = model_end.compute_ntk()
assert(np.allclose(ntk, ntk_loaded))